In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

In [7]:
df = pd.read_csv('./data/Gooddreadbooks/ratings.csv')

In [8]:
# Задание 5.1

# Разбейте данные на обучающую и тестовую выборки в отношении 4:1. В качестве значения параметра random state возьмите число 42.
# Сколько объектов теперь находится в обучающей выборке?

train, test = train_test_split(df, test_size=0.25)